In [4]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding

In [24]:
qd_client = QdrantClient('http://localhost:6333')

In [27]:
import json

with open('documents.json', 'r') as documents_file:
    documents_raw = json.load(documents_file)

documents = []

for course_dict in documents_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [26]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-fag"
EMBEDDING_DIMENSIONALITY = 512

In [ ]:
qd_client.delete(collection_name=collection_name)

In [19]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [20]:
from google import genai
from google.genai import types

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
genai_client = genai.Client()

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    filter = {'course':'data-engineering-zoomcamp'}

    results = index.search(
        query=query,
        filter_dict=filter,
        boost_dict=boost,
        num_results=5
    )

    return results

In [22]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [23]:
def llm(prompt):
    contents = types.Content(
      role='user',
      parts=[types.Part.from_text(text=prompt)]
    )
    
    response = genai_client.models.generate_content(
        model="gemini-2.5-flash", contents=contents
    )

    return response.text

In [10]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer